In [6]:
!pip install gradio

In [2]:
import gradio as gr
import json
import random
from PIL import Image as PILImage

with open('./captioning/double_concept/caption-rap-llava-with-skip-retreival_RAP-llava-reproduce.json', 'r') as f:
    rap = json.load(f)
with open('./captioning/double_concept/caption-rap-llava-skip-retreival_RAP-llava-sft-2k.json', 'r') as f:
    rap_2k = json.load(f)
with open('./captioning/double_concept/captioning-skip-retrieval-ours.json', 'r') as f:
    ours = json.load(f)
with open('./captioning/double_concept/captioning-skip-retrieval-qwen_sft.json', 'r') as f:
    rap_qwen = json.load(f)
with open('./captioning/double_concept/captioning-skip-retrieval-zero_shot.json', 'r') as f:
    zero_shot = json.load(f)
    
database_path = './data/database_tot/database_2_concepts/'
with open(f"{database_path}/database_mod.json", "r") as f:
    database = json.load(f)
    
method_names = ['RAP-LLaVA 13B 210K', 'RAP-LLaVA 13B 2K', 'RAP-Qwen 7B 210K', 'Qwen-2.5VL 7B Zero-Shot', 'Ours 7B']

def random_samples(n=3):
    idx = random.choice(range(len(rap)))
    methods = [rap, rap_2k, rap_qwen, zero_shot, ours]

    img = PILImage.open(rap[idx][0])
    texts = [method[idx][2] for method in methods]    
    query =  [rap[idx][-1]]
    db_samples = rap[idx][1]
    db_imgs = [PILImage.open(database['concept_dict'][s]['image']).convert('RGB').resize((256,256)) for s in db_samples]
    db_texts = [f'Name: {s}, Info: ' + database['concept_dict'][s]['info'] for s in db_samples]

    return [img] + query + texts + db_imgs + db_texts

with gr.Blocks() as demo:
    gr.Markdown("# Random Visualization of Personalized Multi-Concept Image Captioning Examples using MLLM")
    with gr.Row():
        image_output = gr.Image()

    with gr.Row():
        q_textbox = [gr.Textbox(label=f"Query Caption", lines=1)]
 
    with gr.Row():
        text_outputs = [
            gr.Textbox(label=f"Caption {method_names[i]}", lines=5)
            for i in range(5)
        ]
        
    with gr.Row():
        db_images = [
            gr.Image(label=f"DB Image {i+1}") for i in range(2)
        ]

    with gr.Row():
        db_textboxes = [
            gr.Textbox(label=f"DB Caption {i+1}", lines=2)
            for i in range(2)
        ]
    n_samples = gr.Number(value=5, label="Number of Texts", precision=0)
    btn = gr.Button("Pick Random Samples")

    btn.click(
        fn=random_samples,
        inputs=[n_samples],
        outputs=[image_output] + q_textbox + text_outputs + db_images + db_textboxes  
    )
demo.launch(server_name="127.0.0.1", share=False, inbrowser=True)


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
